In [ ]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [1]:
from typing import Generator, List
import pickle

from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
from spacy.lang.en import English

In [2]:
with open(r"../data/interim/review_classes.pkl", "rb") as input_file:
    review_classes = pickle.load(input_file)

In [3]:
review_classes

{'POS': ["This is one of the best hotels I've ever stayed at with incredible staff catering to your every need. I felt like family whilst I was there!   I stayed for 3 weeks during the pandemic and was so impressed with all the measures the hotel have put in place to keep everyone safe and well. They give you a box of masks, gloves and a bottle of hand sanitiser upon arrival, all the staff wear masks and gloves and the rooms and everything else are cleaned to the highest standard.  Plus the prices they've been offering during this time are incredible value for money! Definitely would recommend and hope to be back once travel reopens.   Thank you so much for looking after me so well until I could return home!",
  'Everything about this hotel was awesome. The staff made me feel welcomed from the minute i arrived until the day i departed. The view from the dining area with the mountains in the background was spectacular. The grounds were manicured to perfection with flowering plants in ev

In [4]:
positive_reviews = review_classes['POS']
negative_reviews = review_classes['NEG']

In [5]:
positive_reviews

["This is one of the best hotels I've ever stayed at with incredible staff catering to your every need. I felt like family whilst I was there!   I stayed for 3 weeks during the pandemic and was so impressed with all the measures the hotel have put in place to keep everyone safe and well. They give you a box of masks, gloves and a bottle of hand sanitiser upon arrival, all the staff wear masks and gloves and the rooms and everything else are cleaned to the highest standard.  Plus the prices they've been offering during this time are incredible value for money! Definitely would recommend and hope to be back once travel reopens.   Thank you so much for looking after me so well until I could return home!",
 'Everything about this hotel was awesome. The staff made me feel welcomed from the minute i arrived until the day i departed. The view from the dining area with the mountains in the background was spectacular. The grounds were manicured to perfection with flowering plants in every direc

In [9]:
def sentences_to_words(sentences: List[str]) -> List[List[str]]:
    words = []
    for sentence in sentences:
        # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        words.append(simple_preprocess(str(sentence), deacc=True))  # deacc=True elimina la puntuación
    return words

In [18]:
print(negative_reviews[1])
negative_words = sentences_to_words(negative_reviews)
print(negative_words[1])

We went to the hotel directly to book a room. The receptionist at first gave us a really high price then lowered it. When we asked how much it costs in qetales she gave us an 8.2 comission rate to dollars and I knew she was randomly goving us a non existing rate so then she "lowered" it to the real rate. After we agreed that the price includes breakfast the following morning we get to breakfast and received two plain pieces of toast and fruit while theother tables received also eggs and plantines. So I asked if we also get eggs abd plantines and she replied "no". How come we didn't get the same breakfast when we paid for it?? Not a good experience at all at the hotel. There are much better places in Antigua to stay.
['we', 'went', 'to', 'the', 'hotel', 'directly', 'to', 'book', 'room', 'the', 'receptionist', 'at', 'first', 'gave', 'us', 'really', 'high', 'price', 'then', 'lowered', 'it', 'when', 'we', 'asked', 'how', 'much', 'it', 'costs', 'in', 'qetales', 'she', 'gave', 'us', 'an', 'c

In [19]:
def remove_stopwords(documents: List[List[str]]) -> List[List[str]]:
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords.words('english')]
            for doc in documents]

In [22]:
print(negative_words[0])
negative_words_wo_stopwords = remove_stopwords(negative_words)
print(negative_words_wo_stopwords[0])

['good', 'location', 'on', 'the', 'nw', 'of', 'town', 'especially', 'if', 'youre', 'going', 'to', 'antiguena', 'spanish', 'school', 'for', 'good', 'sized', 'private', 'twin', 'room', 'with', 'shared', 'bathroom', 'great', 'price', 'nice', 'ish', 'garden', 'with', 'swing', 'and', 'hamock', 'basic', 'bathrooms', 'very', 'tatty', 'but', 'has', 'hot', 'showers', 'always', 'if', 'you', 'keep', 'the', 'water', 'flow', 'slow', 'super', 'basic', 'pretty', 'dirty', 'kitchenette', 'loads', 'of', 'cats', 'and', 'dogs', 'its', 'very', 'cheap', 'and', 'basic', 'if', 'thats', 'what', 'your', 'after', 'its', 'ok']
[['good', 'location', 'nw', 'town', 'especially', 'youre', 'going', 'antiguena', 'spanish', 'school', 'good', 'sized', 'private', 'twin', 'room', 'shared', 'bathroom', 'great', 'price', 'nice', 'ish', 'garden', 'swing', 'hamock', 'basic', 'bathrooms', 'tatty', 'hot', 'showers', 'always', 'keep', 'water', 'flow', 'slow', 'super', 'basic', 'pretty', 'dirty', 'kitchenette', 'loads', 'cats', 'd

In [41]:
def learn_bigrams(documents: List[List[str]]) -> List[List[str]]:
    # We learn bigrams
    #  https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
    bigram = Phrases(documents, min_count=5, threshold=10)

    # we reduce the bigram model to its minimal functionality
    bigram_mod = Phraser(bigram)

    # we apply the bigram model to our documents
    return [bigram_mod[doc] for doc in documents]


In [26]:
negative_words_wo_stopwords_bigrams = learn_bigrams(negative_words_wo_stopwords)
print(negative_words_wo_stopwords[0])
print(negative_words_wo_stopwords_bigrams[0])

['good', 'location', 'nw', 'town', 'especially', 'youre', 'going', 'antiguena', 'spanish', 'school', 'good', 'sized', 'private', 'twin', 'room', 'shared', 'bathroom', 'great', 'price', 'nice', 'ish', 'garden', 'swing', 'hamock', 'basic', 'bathrooms', 'tatty', 'hot', 'showers', 'always', 'keep', 'water', 'flow', 'slow', 'super', 'basic', 'pretty', 'dirty', 'kitchenette', 'loads', 'cats', 'dogs', 'cheap', 'basic', 'thats', 'ok']
['good_location', 'nw', 'town', 'especially', 'youre', 'going', 'antiguena', 'spanish_school', 'good', 'sized', 'private', 'twin', 'room', 'shared_bathroom', 'great', 'price', 'nice', 'ish', 'garden', 'swing', 'hamock', 'basic', 'bathrooms', 'tatty', 'hot', 'showers', 'always', 'keep', 'water', 'flow', 'slow', 'super', 'basic', 'pretty', 'dirty', 'kitchenette', 'loads', 'cats', 'dogs', 'cheap', 'basic', 'thats', 'ok']


In [29]:
def lemmatization(nlp: English, texts: List[List[str]], allowed_postags: List = None) -> List[List[str]]:
    if allowed_postags is None:
        allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']

    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [30]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

In [31]:
negative_words_wo_stopwords_bigrams_pos = lemmatization(nlp, negative_words_wo_stopwords_bigrams)

['good_location', 'nw', 'town', 'especially', 'youre', 'going', 'antiguena', 'spanish_school', 'good', 'sized', 'private', 'twin', 'room', 'shared_bathroom', 'great', 'price', 'nice', 'ish', 'garden', 'swing', 'hamock', 'basic', 'bathrooms', 'tatty', 'hot', 'showers', 'always', 'keep', 'water', 'flow', 'slow', 'super', 'basic', 'pretty', 'dirty', 'kitchenette', 'loads', 'cats', 'dogs', 'cheap', 'basic', 'thats', 'ok']
['town', 'especially', 'be', 'go', 'antiguena', 'good', 'sized', 'private', 'twin', 'room', 'shared_bathroom', 'great', 'price', 'nice', 'ish', 'garden', 'swing', 'basic', 'bathroom', 'tatty', 'hot', 'shower', 'always', 'keep', 'water', 'flow', 'slow', 'super', 'basic', 'pretty', 'dirty', 'kitchenette', 'load', 'cat', 'dog', 'cheap', 'basic', 's']


In [34]:
print(negative_words_wo_stopwords_bigrams[2])
print(negative_words_wo_stopwords_bigrams_pos[2])

['going', 'break', 'good', 'bad', 'good_location', 'free_breakfast', 'hot_shower', 'warm', 'helpful', 'staff', 'one', 'hour', 'massage', 'bad', 'small', 'rooms', 'without', 'great', 'air_circulation', 'awful', 'quality', 'beds', 'noisy', 'dogs', 'next_door', 'let', 'decide', 'important', 'awful', 'bed', 'kills', 'massages', 'made', 'feel', 'better', 'choice', 'sure', 'come_back', 'back', 'handle', 'way', 'night', 'advertised', 'totally', 'wrong', 'ended', 'night', 'triple', 'room', 'really', 'worth', 'since', 'many', 'choices']
['go', 'break', 'good', 'bad', 'good_location', 'free_breakfast', 'warm', 'helpful', 'staff', 'hour', 'massage', 'bad', 'small', 'room', 'great', 'air_circulation', 'awful', 'quality', 'bed', 'noisy', 'dog', 'let', 'decide', 'important', 'awful', 'bed', 'kill', 'massage', 'make', 'feel', 'well', 'choice', 'sure', 'come_back', 'back', 'handle', 'way', 'night', 'advertise', 'totally', 'wrong', 'end', 'night', 'triple', 'room', 'really', 'worth', 'many', 'choice']


In [42]:
def tokenize(documents: List[str]) -> List[List[str]]:

    document_words = sentences_to_words(documents)
    document_words = remove_stopwords(document_words)
    document_words = learn_bigrams(document_words)
    document_words = lemmatization(nlp, document_words)

    return document_words

In [43]:
positive_words = tokenize(positive_reviews)

In [44]:
negative_words = tokenize(negative_reviews)

In [39]:
print(positive_words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
with open(r"../data/interim/positive_words.pkl", "wb") as output_file:
    pickle.dump(positive_words, output_file)

with open(r"../data/interim/negative_words.pkl", "wb") as output_file:
    pickle.dump(negative_words, output_file)